# Simple GRU network with pretrained vectors for initialization

In [1]:
import sys, os, re, csv, codecs, gc, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

from Tokenizer import Tokenizer
from ZeroMaskedLayer import ZeroMaskedLayer
from AttentionLayer import AttentionLayer

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{utility_path}glove.42B.300d.txt'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
word_tokenize("I won't do this check!!!")

['I', 'wo', "n't", 'do', 'this', 'check', '!', '!', '!']

In [4]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [5]:
def unicodeToAscii(series):
    return series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))


def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))

    def one_xlat(match):
        return adict[match.group(0)]

    return rx.sub(one_xlat, text)

STOP_WORDS = set(stopwords.words( 'english' ))
# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)
    series = series.str.lower()
    series = series.str.replace(r"(\n){1,}", " ")
    #series = series.str.replace(r"\'", "")
    #series = series.str.replace(r"\-", "")
    #series = series.str.replace(r"[^0-9a-zA-Z.,!?]+", " ")

    return series


In [6]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [7]:
for df in train, test:
    df["comment_text"] = normalizeString(df["comment_text"])

In [8]:
train.comment_text.sample(1).values[0]

'i will test alternate summaries for greater clarity.'

In [9]:
#pd.concat([train["comment_text"].astype(str), test["comment_text"].astype(str)]).reset_index(drop=True)[:len(train), :]

In [10]:
MAX_FEATURES = 250000
MAX_LEN = 250

tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN, tokenizer=word_tokenize)
X = tok.fit_transform(pd.concat([train["comment_text"].astype(str), test["comment_text"].astype(str)]))
X_train = X[:len(train), :]
X_test = X[len(train):, :]

print(X_train.shape, X_test.shape)

(159571, 250) (153164, 250)


In [11]:
#del train, test

In [19]:
EMBED_SIZE = 300
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

def initialize_embeddings(filename, tokenizer):
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(filename))

    word_index = tokenizer.vocab_idx
    nb_words = min(MAX_FEATURES+1, len(word_index)+1)
    embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
    for word, i in word_index.items():
        if i > MAX_FEATURES: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
embedding_matrix = initialize_embeddings(EMBEDDING_FILE, tok)
print(embedding_matrix.shape)
print(np.mean(embedding_matrix), np.std(embedding_matrix))

In [14]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=6, callbacks=None,
                spatial_drop=0.0, dropout=0.0, mask_zero=True, 
                gru_kernel_regularization = 0.0,
                gru_recurrent_regularization = 0.0,
                gru_bias_regularization = 0.0,
                embeddings_regularization = 0.0,
                ):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
        self.spatial_drop = spatial_drop
        self.dropout = dropout
        self.mask_zero = mask_zero
        self.gru_kernel_regularization = gru_kernel_regularization
        self.gru_recurrent_regularization = gru_recurrent_regularization
        self.gru_bias_regularization = gru_bias_regularization
        self.embeddings_regularization = embeddings_regularization
        
    def _build_model(self):
        inp = Input(shape=(MAX_LEN,))
        emb = Embedding(MAX_FEATURES+1, 
                        EMBED_SIZE,
                        weights=[self.initial_weights],
                        mask_zero=self.mask_zero,
                        #embeddings_regularizer=regularizers.l2(self.embeddings_regularization),
                        trainable=True)(inp)

        if self.mask_zero:
            emb = ZeroMaskedLayer()(emb)
            
        emb = SpatialDropout1D(self.spatial_drop)(emb)
        if self.bidirectional:
            enc = Bidirectional(GRU(int(self.gru_dim), return_sequences=True, return_state=True, stateful=True,
                                         ))(emb)
            x = enc[0]
            state = enc[1]
        else:
            x, state = GRU(int(self.gru_dim), return_sequences=True, return_state=True,
                            kernel_regularizer=regularizers.l2(self.gru_kernel_regularization),
                            recurrent_regularizer=regularizers.l2(self.gru_recurrent_regularization),
                            bias_regularizer=regularizers.l2(self.gru_bias_regularization)
                               )(emb)
            #x = SpatialDropout1D(0.5)(x)
        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            #x1 = GlobalAveragePooling1D()(x)
            x2 = GlobalMaxPool1D()(x)
            x3 = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x2, x3, state])
    
        x = Dropout(self.dropout)(x)
        x = Dense(self.dense_dim)(x)
        x = PReLU()(x)
        
        #x = Dense(self.dense_dim)(x)
        #x = PReLU()(x)

        out = Dense(self.out_dim, activation="sigmoid")(x)
        if self.optimizer == 'adam':
            opt = Adam(lr=0.001, decay=0.0, clipnorm=1.0)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(clipnorm=1.0)
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=1024)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [15]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.0016
    if epoch == 1:
        return 0.00024
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.00001


def shuffle_crossvalidator(model, cvlist, X, y, lr_decay):
    y_trues = []
    y_preds = []
    scores = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    score = roc_auc_score(y_trues, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, scores

def outoffold_crossvalidator(model_params, cvlist, X, y, lr_decay):
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        
        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_preds[val_idx] = y_pred
        K.clear_session()
        break
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score


In [17]:
import tensorflow as tf
K.clear_session()
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
sess = tf.Session(config=config)
model = GRUClassifier(gru_dim=300, dense_dim=900, initial_weights=embedding_matrix, bidirectional=False,
                    batch_size=64, epochs=2, optimizer='adam', pool_type='all', dropout=0.2, spatial_drop=0.3, mask_zero=False)

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y, lr_decay)

Epoch 1/2
 42368/151592 [=======>......................] - ETA: 42:58 - loss: 0.0627 - acc: 0.9781

InvalidArgumentError: indices[53,110] = 250000 is not in [0, 250000)
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast)]]

Caused by op 'embedding_1/Gather', defined at:
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/

In [15]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


def uniform_int(name, lower, upper):
    # `quniform` returns:
    # round(uniform(low, high) / q) * q
    return hp.quniform(name, lower, upper, q=1)

def loguniform_int(name, lower, upper):
    # Do not forget to make a logarithm for the
    # lower and upper bounds.
    return hp.qloguniform(name, np.log(lower), np.log(upper), q=1)

parameter_space = {
    'gru_dim': uniform_int('gru_dim', 50, 600),
    'dense_dim': uniform_int('dense_dim', 100, 1000),
    'lr1': hp.uniform('lr1', 0.0001, 0.005),
    'lr2': hp.uniform('lr2', 0.0001, 0.005),
    'spatial_drop': hp.uniform('spatial_drop', 0, 0.5),
    'dropout': hp.uniform('dropout', 0, 0.5),
    'batch_size': loguniform_int('batch_size', 16, 512),
    'mask_zero': hp.choice('mask_zero', [True, False]),
    'optimizer': hp.choice('optimizer', ['adam', 'rmsprop']),
    'pool_type': hp.choice('pool_type', ['avg', 'max', 'attn', 'all']),
    'bidirectional': hp.choice('bidirectional', [True, False]),
    'gru_kernel_reg': hp.loguniform('gru_kernel_reg', np.log(1e-10), np.log(1e-4)),
    'gru_recc_reg': hp.loguniform('gru_recc_reg', np.log(1e-10), np.log(1e-4)),
    'gru_bias_reg': hp.loguniform('gru_bias_reg', np.log(1e-10), np.log(1e-4)),
    #'embeddings_reg': hp.loguniform('embeddings_reg', 1e-8, 1e-4)
}


def objective(parameter_space):
    
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1']
        if epoch == 1:
            return parameter_space['lr2']
    
    model = GRUClassifier(initial_weights=embedding_matrix, bidirectional=False,
                          gru_dim = int(parameter_space['gru_dim']),
                          dense_dim = int(parameter_space['dense_dim']),
                          mask_zero = parameter_space['mask_zero'],
                          pool_type = parameter_space['pool_type'],
                          batch_size= int(parameter_space['batch_size']), 
                          epochs=2, 
                          optimizer=parameter_space['optimizer'],
                          dropout=parameter_space['dropout'],
                          spatial_drop=parameter_space['spatial_drop'],
                          gru_kernel_regularization = parameter_space["gru_kernel_reg"],
                          gru_recurrent_regularization = parameter_space["gru_recc_reg"],
                          gru_bias_regularization = parameter_space["gru_bias_reg"],
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, scores = shuffle_crossvalidator(model, cvlist1, X_train, y, lr_decay)    
    score = roc_auc_score(y_trues, y_preds)
    print("Score for parameters {} is {}".format(parameter_space, score))
    #return score
    return {
        'loss': -1* score,
        'status': STATUS_OK,
        'other_Stuff': {'scores': scores, 'variance': np.std(scores)},
        }

trials = Trials()

best = fmin(objective,
    space=parameter_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
           )

print(best)

Epoch 1/2
151524/151592 [============================>.] - ETA: 0s - loss: 0.0553 - acc: 0.9797
 ROC-AUC - epoch: 1 - score: 0.983718 

151592/151592 [==============================] - 65s 431us/step - loss: 0.0553 - acc: 0.9797
Epoch 2/2
151524/151592 [============================>.] - ETA: 0s - loss: 0.0366 - acc: 0.9854
 ROC-AUC - epoch: 2 - score: 0.987400 

151592/151592 [==============================] - 65s 430us/step - loss: 0.0366 - acc: 0.9854
ROC AUC for this fold is  0.9874001190566956
Epoch 1/2
151524/151592 [============================>.] - ETA: 0s - loss: 0.0556 - acc: 0.9794
 ROC-AUC - epoch: 1 - score: 0.985117 

151592/151592 [==============================] - 65s 430us/step - loss: 0.0556 - acc: 0.9794
Epoch 2/2
151524/151592 [============================>.] - ETA: 0s - loss: 0.0351 - acc: 0.9860
 ROC-AUC - epoch: 2 - score: 0.986998 

151592/151592 [==============================] - 65s 431us/step - loss: 0.0350 - acc: 0.9860
ROC AUC for this fold is  0.98699770707

Epoch 2/2
151305/151592 [============================>.] - ETA: 0s - loss: 0.0521 - acc: 0.9818
 ROC-AUC - epoch: 2 - score: 0.984349 

151592/151592 [==============================] - 48s 320us/step - loss: 0.0521 - acc: 0.9818
ROC AUC for this fold is  0.9843494135717098
Overall score on 10 fold CV is 0.984173108000812
Score for parameters {'batch_size': 393.0, 'bidirectional': False, 'dense_dim': 787.0, 'dropout': 0.28724271025408527, 'gru_bias_reg': 2.476305113127399e-10, 'gru_dim': 370.0, 'gru_kernel_reg': 1.343646120172816e-06, 'gru_recc_reg': 1.5632252487944346e-06, 'lr1': 0.002611331429648933, 'lr2': 0.003986991693636472, 'mask_zero': False, 'optimizer': 'rmsprop', 'pool_type': 'attn', 'spatial_drop': 0.12151886966722963} is 0.984173108000812
Epoch 1/2
151515/151592 [============================>.] - ETA: 0s - loss: 0.0578 - acc: 0.9802
 ROC-AUC - epoch: 1 - score: 0.981466 

151592/151592 [==============================] - 172s 1ms/step - loss: 0.0578 - acc: 0.9802
Epoch 2/2
1

ROC AUC for this fold is  0.4676613898967063
Epoch 1/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.5906 - acc: 0.9628
 ROC-AUC - epoch: 1 - score: 0.618471 

151592/151592 [==============================] - 80s 530us/step - loss: 0.5905 - acc: 0.9629
Epoch 2/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.5910 - acc: 0.9633
 ROC-AUC - epoch: 2 - score: 0.616088 

151592/151592 [==============================] - 80s 529us/step - loss: 0.5909 - acc: 0.9633
ROC AUC for this fold is  0.6160864161293582
Epoch 1/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.5903 - acc: 0.9630
 ROC-AUC - epoch: 1 - score: 0.577216 

151592/151592 [==============================] - 80s 529us/step - loss: 0.5902 - acc: 0.9630
Epoch 2/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.5909 - acc: 0.9633
 ROC-AUC - epoch: 2 - score: 0.559668 

151592/151592 [==============================] - 80s 528us/step - loss: 0.5909 - acc: 0

Epoch 2/2
151360/151592 [============================>.] - ETA: 0s - loss: 0.5907 - acc: 0.9634
 ROC-AUC - epoch: 2 - score: 0.500000 

151592/151592 [==============================] - 61s 401us/step - loss: 0.5910 - acc: 0.9633
ROC AUC for this fold is  0.5
Epoch 1/2
151360/151592 [============================>.] - ETA: 0s - loss: 0.0601 - acc: 0.9800
 ROC-AUC - epoch: 1 - score: 0.984129 

151592/151592 [==============================] - 61s 403us/step - loss: 0.0601 - acc: 0.9800
Epoch 2/2
151360/151592 [============================>.] - ETA: 0s - loss: 0.0384 - acc: 0.9858
 ROC-AUC - epoch: 2 - score: 0.986601 

151592/151592 [==============================] - 61s 405us/step - loss: 0.0384 - acc: 0.9858
ROC AUC for this fold is  0.9866005135660041
Epoch 1/2
151360/151592 [============================>.] - ETA: 0s - loss: 0.0613 - acc: 0.9804
 ROC-AUC - epoch: 1 - score: 0.983569 

151592/151592 [==============================] - 61s 401us/step - loss: 0.0613 - acc: 0.9804
Epoch 2/2

Epoch 1/2
151560/151592 [============================>.] - ETA: 0s - loss: 0.0686 - acc: 0.9779
 ROC-AUC - epoch: 1 - score: 0.971304 

151592/151592 [==============================] - 390s 3ms/step - loss: 0.0686 - acc: 0.9779
Epoch 2/2
151578/151592 [============================>.] - ETA: 0s - loss: 0.0642 - acc: 0.9797
 ROC-AUC - epoch: 2 - score: 0.943943 

151592/151592 [==============================] - 389s 3ms/step - loss: 0.0642 - acc: 0.9797
ROC AUC for this fold is  0.943952179897605
Epoch 1/2
151560/151592 [============================>.] - ETA: 0s - loss: 0.0727 - acc: 0.9768
 ROC-AUC - epoch: 1 - score: 0.969941 

151592/151592 [==============================] - 392s 3ms/step - loss: 0.0727 - acc: 0.9769
Epoch 2/2
151560/151592 [============================>.] - ETA: 0s - loss: 0.0708 - acc: 0.9783
 ROC-AUC - epoch: 2 - score: 0.957174 

151592/151592 [==============================] - 391s 3ms/step - loss: 0.0708 - acc: 0.9783
ROC AUC for this fold is  0.9571949727861832

...

Epoch 1/2
151551/151592 [============================>.] - ETA: 0s - loss: 0.0509 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.986512 

151592/151592 [==============================] - 130s 857us/step - loss: 0.0509 - acc: 0.9814
Epoch 2/2
151551/151592 [============================>.] - ETA: 0s - loss: 0.0324 - acc: 0.9870
 ROC-AUC - epoch: 2 - score: 0.988896 

151592/151592 [==============================] - 130s 857us/step - loss: 0.0324 - acc: 0.9870
ROC AUC for this fold is  0.988896066951209
Epoch 1/2
151551/151592 [============================>.] - ETA: 0s - loss: 0.0509 - acc: 0.9812
 ROC-AUC - epoch: 1 - score: 0.987651 

151592/151592 [==============================] - 130s 856us/step - loss: 0.0509 - acc: 0.9812
Epoch 2/2
151551/151592 [============================>.] - ETA: 0s - loss: 0.0324 - acc: 0.9869
 ROC-AUC - epoch: 2 - score: 0.988204 

151592/151592 [==============================] - 130s 857us/step - loss: 0.0324 - acc: 0.9869
ROC AUC for this fold is  0.98820402

Epoch 2/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.0392 - acc: 0.9851
 ROC-AUC - epoch: 2 - score: 0.986129 

151592/151592 [==============================] - 122s 807us/step - loss: 0.0392 - acc: 0.9851
ROC AUC for this fold is  0.9861294244791637
Overall score on 10 fold CV is 0.9854291597372593
Score for parameters {'batch_size': 88.0, 'bidirectional': True, 'dense_dim': 117.0, 'dropout': 0.10946398086755264, 'gru_bias_reg': 2.2807694853841946e-09, 'gru_dim': 479.0, 'gru_kernel_reg': 2.9486712768962895e-09, 'gru_recc_reg': 7.404183757908396e-08, 'lr1': 0.0025093618137330335, 'lr2': 0.0007617646397483003, 'mask_zero': True, 'optimizer': 'rmsprop', 'pool_type': 'all', 'spatial_drop': 0.19149226919902412} is 0.9854291597372593
Epoch 1/2
151575/151592 [============================>.] - ETA: 0s - loss: 0.0516 - acc: 0.9815
 ROC-AUC - epoch: 1 - score: 0.987315 

151592/151592 [==============================] - 105s 690us/step - loss: 0.0516 - acc: 0.9815
Epoch 2

ROC AUC for this fold is  0.9862165475838536
Epoch 1/2
151515/151592 [============================>.] - ETA: 0s - loss: 0.0532 - acc: 0.9809
 ROC-AUC - epoch: 1 - score: 0.985907 

151592/151592 [==============================] - 126s 832us/step - loss: 0.0532 - acc: 0.9809
Epoch 2/2
151515/151592 [============================>.] - ETA: 0s - loss: 0.0353 - acc: 0.9860
 ROC-AUC - epoch: 2 - score: 0.987113 

151592/151592 [==============================] - 126s 833us/step - loss: 0.0353 - acc: 0.9860
ROC AUC for this fold is  0.9871134566341603
Epoch 1/2
151515/151592 [============================>.] - ETA: 0s - loss: 0.0534 - acc: 0.9809
 ROC-AUC - epoch: 1 - score: 0.983645 

151592/151592 [==============================] - 126s 832us/step - loss: 0.0534 - acc: 0.9809
Epoch 2/2
151515/151592 [============================>.] - ETA: 0s - loss: 0.0359 - acc: 0.9857
 ROC-AUC - epoch: 2 - score: 0.986974 

151592/151592 [==============================] - 126s 831us/step - loss: 0.0360 - ac

Epoch 2/2
151578/151592 [============================>.] - ETA: 0s - loss: 0.0336 - acc: 0.9872
 ROC-AUC - epoch: 2 - score: 0.986424 

151592/151592 [==============================] - 178s 1ms/step - loss: 0.0336 - acc: 0.9872
ROC AUC for this fold is  0.9864241193579635
Epoch 1/2
151578/151592 [============================>.] - ETA: 0s - loss: 0.0520 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.985446 

151592/151592 [==============================] - 179s 1ms/step - loss: 0.0520 - acc: 0.9814
Epoch 2/2
151578/151592 [============================>.] - ETA: 0s - loss: 0.0333 - acc: 0.9873
 ROC-AUC - epoch: 2 - score: 0.987373 

151592/151592 [==============================] - 179s 1ms/step - loss: 0.0334 - acc: 0.9873
ROC AUC for this fold is  0.9873728019955531
Epoch 1/2
151578/151592 [============================>.] - ETA: 0s - loss: 0.0526 - acc: 0.9812
 ROC-AUC - epoch: 1 - score: 0.987587 

151592/151592 [==============================] - 178s 1ms/step - loss: 0.0526 - acc: 0.981

Epoch 1/2
151569/151592 [============================>.] - ETA: 0s - loss: 0.0597 - acc: 0.9810
 ROC-AUC - epoch: 1 - score: 0.980846 

151592/151592 [==============================] - 218s 1ms/step - loss: 0.0597 - acc: 0.9810
Epoch 2/2
151569/151592 [============================>.] - ETA: 0s - loss: 0.0537 - acc: 0.9830
 ROC-AUC - epoch: 2 - score: 0.982288 

151592/151592 [==============================] - 218s 1ms/step - loss: 0.0537 - acc: 0.9830
ROC AUC for this fold is  0.9822882533445307
Epoch 1/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.0614 - acc: 0.9807
 ROC-AUC - epoch: 1 - score: 0.979426 

151592/151592 [==============================] - 220s 1ms/step - loss: 0.0614 - acc: 0.9807
Epoch 2/2
151569/151592 [============================>.] - ETA: 0s - loss: 0.0534 - acc: 0.9830
 ROC-AUC - epoch: 2 - score: 0.978691 

151592/151592 [==============================] - 220s 1ms/step - loss: 0.0534 - acc: 0.9830
ROC AUC for this fold is  0.978690826382508

In [16]:
import gc 
gc.collect()
K.clear_session()

In [36]:
top10 = sorted(trials.trials, key=lambda x: x['result']['loss'])[:10]

parameter_list =[tr['misc']['vals'] for tr in top10]
parameter_list

[{'batch_size': [47.0],
  'bidirectional': [1],
  'dense_dim': [973.0],
  'dropout': [0.19862535182199834],
  'gru_bias_reg': [1.011936859273273e-08],
  'gru_dim': [358.0],
  'gru_kernel_reg': [2.0678669679829352e-10],
  'gru_recc_reg': [8.946942716621634e-07],
  'lr1': [0.0015982451490776767],
  'lr2': [0.0002459290205687559],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.2696100622336198]},
 {'batch_size': [83.0],
  'bidirectional': [1],
  'dense_dim': [351.0],
  'dropout': [0.07833431778315075],
  'gru_bias_reg': [1.989216237371643e-09],
  'gru_dim': [478.0],
  'gru_kernel_reg': [2.1606860352426398e-10],
  'gru_recc_reg': [1.6736919208281796e-07],
  'lr1': [0.00263784102869703],
  'lr2': [0.0005711207564167526],
  'mask_zero': [1],
  'optimizer': [0],
  'pool_type': [3],
  'spatial_drop': [0.21401382410917008]},
 {'batch_size': [49.0],
  'bidirectional': [1],
  'dense_dim': [997.0],
  'dropout': [0.19115533803668047],
  'gru_bias_reg': [5.222640591

In [ ]:
#Pick top 10 parameter settings, Bag models for those settings
#Try linear blending on those settings
NUM_BAGS = 10
cvlist3 = list(StratifiedShuffleSplit(n_splits=NUM_BAGS, test_size=0.05, random_state=786).split(y, y[:,2]))


def shuffle_train_predict(model, cvlist, X, y, X_test, lr_decay):
    y_trues = []
    y_preds = []
    y_test_preds = []
    scores = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        y_test_preds.append(y_test_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    y_test_preds = np.mean(y_test_preds, axis=0)
    print("Shape of test _preds is ", y_test_preds.shape)
    print("Means of val and test preds are {} and {}".format(np.mean(y_preds, axis=1), np.mean(y_test_preds, axis=1)))
    score = roc_auc_score(y_trues, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, y_test_preds

def train_predict(parameter_space):
    
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1'][0]
        if epoch == 1:
            return parameter_space['lr2'][0]
    
    model = GRUClassifier(initial_weights=embedding_matrix, bidirectional=[True, False][parameter_space['bidirectional'][0]],
                          gru_dim = int(parameter_space['gru_dim'][0]),
                          dense_dim = int(parameter_space['dense_dim'][0]),
                          mask_zero = [True, False][parameter_space['mask_zero'][0]],
                          pool_type = ['avg', 'max', 'attn', 'all'][parameter_space['pool_type'][0]],
                          batch_size= int(parameter_space['batch_size'][0]), 
                          epochs=2, 
                          optimizer=["adam", "rmsprop"][parameter_space['optimizer'][0]],
                          dropout=parameter_space['dropout'][0],
                          spatial_drop=parameter_space['spatial_drop'][0],
                          gru_kernel_regularization = parameter_space["gru_kernel_reg"][0],
                          gru_recurrent_regularization = parameter_space["gru_recc_reg"][0],
                          gru_bias_regularization = parameter_space["gru_bias_reg"][0],
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, y_test_preds = shuffle_train_predict(model, cvlist3, X_train, y, X_test, lr_decay)    
    return y_preds, y_trues, y_test_preds

#####
y_preds_all = []
y_trues_all = []
y_test_preds_all = []
for params in parameter_list:
    y_preds, y_trues, y_test_preds = train_predict(params)
    y_preds_all.append(y_preds)
    y_trues_all.append(y_trues)
    y_test_preds_all.append(y_test_preds)
    


Epoch 1/2
151575/151592 [============================>.] - ETA: 0s - loss: 0.0510 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.989692 

151592/151592 [==============================] - 186s 1ms/step - loss: 0.0510 - acc: 0.9814
Epoch 2/2
151575/151592 [============================>.] - ETA: 0s - loss: 0.0344 - acc: 0.9863
 ROC-AUC - epoch: 2 - score: 0.990572 

151592/151592 [==============================] - 186s 1ms/step - loss: 0.0344 - acc: 0.9863
ROC AUC for this fold is  0.9905723708910106
Epoch 1/2
151575/151592 [============================>.] - ETA: 0s - loss: 0.0505 - acc: 0.9816
 ROC-AUC - epoch: 1 - score: 0.986022 

151592/151592 [==============================] - 186s 1ms/step - loss: 0.0505 - acc: 0.9816
Epoch 2/2
151575/151592 [============================>.] - ETA: 0s - loss: 0.0340 - acc: 0.9865
 ROC-AUC - epoch: 2 - score: 0.988035 

151592/151592 [==============================] - 185s 1ms/step - loss: 0.0340 - acc: 0.9865
ROC AUC for this fold is  0.988035033127364

ROC AUC for this fold is  0.99035323100561
Epoch 1/2
151558/151592 [============================>.] - ETA: 0s - loss: 0.0500 - acc: 0.9816
 ROC-AUC - epoch: 1 - score: 0.988255 

151592/151592 [==============================] - 141s 927us/step - loss: 0.0500 - acc: 0.9816
Epoch 2/2
151558/151592 [============================>.] - ETA: 0s - loss: 0.0322 - acc: 0.9870
 ROC-AUC - epoch: 2 - score: 0.989766 

151592/151592 [==============================] - 140s 922us/step - loss: 0.0322 - acc: 0.9870
ROC AUC for this fold is  0.9897663028885799
Epoch 1/2
151558/151592 [============================>.] - ETA: 0s - loss: 0.0500 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.985113 

151592/151592 [==============================] - 140s 926us/step - loss: 0.0500 - acc: 0.9814
Epoch 2/2
151558/151592 [============================>.] - ETA: 0s - loss: 0.0320 - acc: 0.9871
 ROC-AUC - epoch: 2 - score: 0.987764 

151592/151592 [==============================] - 138s 913us/step - loss: 0.0320 - acc:

151577/151592 [============================>.] - ETA: 0s - loss: 0.0517 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.985382 

151592/151592 [==============================] - 190s 1ms/step - loss: 0.0517 - acc: 0.9814
Epoch 2/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.0358 - acc: 0.9861
 ROC-AUC - epoch: 2 - score: 0.987807 

151592/151592 [==============================] - 182s 1ms/step - loss: 0.0358 - acc: 0.9861
ROC AUC for this fold is  0.9878066676398046
Epoch 1/2
151577/151592 [============================>.] - ETA: 0s - loss: 0.0515 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.988377 

151592/151592 [==============================] - 181s 1ms/step - loss: 0.0515 - acc: 0.9814
Epoch 2/2
151536/151592 [============================>.] - ETA: 0s - loss: 0.0358 - acc: 0.9861
 ROC-AUC - epoch: 2 - score: 0.989610 

151592/151592 [==============================] - 182s 1ms/step - loss: 0.0358 - acc: 0.9861
ROC AUC for this fold is  0.9896096401234588
Epoch 1/

In [47]:
#Check corelation between different predictions
np.corrcoef(y_preds_all, axis=0)

0

In [ ]:
#Try different stacking approaches
from scipy.stats import gmean, hmean

preds_mean = gmean(y_preds_all, axis=0)
print(roc_auc_score(y_trues_all[0], preds_mean))
test_preds_mean = gmean(y_test_preds_all, axis=0)


In [ ]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[label_cols] = test_preds_mean
sample_submission.to_csv('../input/gru_fasttext_10bags_submission.csv', index=False)